# The PageRank Algorithm

### A simple directed graph

In [1]:
import numpy as np

In [2]:
class Graph:
    def __init__(self, A = None, p = None):
        
        #Shape of the matrix
        A = np.array(A)
        a, b = A.shape
        if a != b:
            raise ValueError('Please give an n times n matrix!')
        else:
            pass
        
        #Transforming the matrix
        for i in np.arange(len(A)):
            if (A[:, i] == np.zeros(len(A))).all():
                A[:, i] = np.ones(len(A))
            else:
                pass
        
        self.A = A
        
        #Label length error
        if p != None and len(p) != len(A):
            raise ValueError('Please specify a label list that has the same length as the matrix!')
        else:
            pass
        
        #Default p value
        if p == None:
            self.p = [str(i) for i in np.arange(len(A))]
        else:
            self.p = [str(i) for i in p]

In [3]:
A = [[0, 0, 0, 0], [1, 0 , 1, 0], [1, 0, 0, 1], [1, 0, 1, 0]]

In [4]:
graph = Graph(A)

In [5]:
graph.A

array([[0, 1, 0, 0],
       [1, 1, 1, 0],
       [1, 1, 0, 1],
       [1, 1, 1, 0]])

In [6]:
graph.p

['0', '1', '2', '3']

### Adding methods to the directed graph class

In [7]:
import scipy.linalg as la

In [8]:
class Graph:
    def __init__(self, A = None, p = None, e = 0.85):
        
        self.e = e
        
        #Shape of the matrix
        A = np.array(A)
        a, b = A.shape
        if a != b:
            raise ValueError('Please give an n times n matrix!')
        else:
            pass
        
        #Transforming the matrix
        for i in np.arange(len(A)):
            if (A[:, i] == np.zeros(len(A))).all():
                A[:, i] = np.ones(len(A))
            else:
                pass
        
        #Normalizing
        A_hat = np.zeros([len(A), len(A)])
        for i in np.arange(len(A)):
            A_hat[:, i] = A[:, i]/np.sum(A[:, i])
        
        self.A = A_hat
        
        #Label length error
        if p != None and len(p) != len(A):
            raise ValueError('Please specify a label list that has the same length as the matrix!')
        else:
            pass
        
        #Default p value
        if p == None:
            self.p = [str(i) for i in np.arange(len(A))]
        else:
            self.p = [str(i) for i in p]
            
    def linsolve(self):
            lvals = la.solve(np.eye(len(self.A)) - self.e * self.A, (1-self.e)/len(self.A) * np.ones(len(self.A)))
            return dict(zip(self.p, lvals))
        
    def eigensolve(self):
            vals, vecs = np.linalg.eig(self.e * self.A + (1-self.e)/len(self.A) * np.ones([len(self.A), len(self.A)]))
            ind = list(vals).index(max(abs(vals)))
            return dict(zip(self.p, vecs[:, ind]/np.sum(vecs[:, ind])))
        
    def itersolve(self, maxiter, tol):
        p0 = np.array([1/len(A)] *len(A))
        B = np.array(self.e * self.A + (1-self.e)/len(self.A) * np.ones([len(self.A), len(self.A)]))
        for i in np.arange(maxiter):
            pt = np.matmul(B, p0)
            if np.linalg.norm(pt - p0, 1) < tol:
                break
            else:
                pass
            p0 = pt
        return dict(zip(self.p, pt))

In [9]:
A = [[0, 0, 0, 0], [1, 0 , 1, 0], [1, 0, 0, 1], [1, 0, 1, 0]]

In [10]:
graph = Graph(A, ['a', 'b', 'c', 'd'])

In [11]:
graph.A

array([[0.        , 0.25      , 0.        , 0.        ],
       [0.33333333, 0.25      , 0.5       , 0.        ],
       [0.33333333, 0.25      , 0.        , 1.        ],
       [0.33333333, 0.25      , 0.5       , 0.        ]])

In [12]:
graph.linsolve() #OK!

{'a': 0.09575863576738088,
 'b': 0.27415828596414527,
 'c': 0.355924792304329,
 'd': 0.27415828596414527}

In [13]:
graph.eigensolve() #OK!

{'a': 0.09575863576738093,
 'b': 0.27415828596414504,
 'c': 0.3559247923043289,
 'd': 0.27415828596414515}

In [14]:
graph.itersolve(1000, 0.004) #OK!

{'a': 0.09588256064264995,
 'b': 0.27381646289116346,
 'c': 0.356484513575023,
 'd': 0.27381646289116346}

### Let's rank the pages!

In [15]:
def p_sorter(d: dict):
    return list({k: v for k, v in sorted(d.items(), key=lambda item: item[1])}.keys())

In [16]:
p_sorter(graph.linsolve()) #OK!

['a', 'b', 'd', 'c']

### Let's find the most visited pages!

In [17]:
import pandas as pd

In [18]:
data = {}
with open('web_stanford.txt') as fh:
    for line in fh:
        values = line.split("/")
        values[-1] = values[-1].split("\n")[0]
        data[values[0]] = values[1:]

In [19]:
sites = list(data.keys())

In [20]:
network = {}
for u, j in enumerate(data.values()):
    row = [0]*len(sites)
    for k in j:
        if k in sites:
            row[sites.index(str(k))] = 1
        else:
            pass
    network[sites[u]] = row

In [21]:
network = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in network.items() ]))
network # Check!

,332,246911,202286,194898,193160,197549,202150,132956,117434,111873,...,188707,146603,147255,110923,102329,244703,189051,151789,127675,99974
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
621,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
622,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
623,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
network = np.array(network.values)

In [23]:
network.shape

(625, 625)

In [24]:
len(sites)

625

In [25]:
#Finding the most visited pages
nw = Graph(network, sites)

In [26]:
p_sorter(nw.linsolve())[-3:] #TOP3 (3rd-2nd-1st)

['28392', '32791', '98595']

### Finding top actors

In [27]:
import networkx as nx
from itertools import combinations

In [28]:
data = []
with open('top250movies.txt', encoding='utf-8') as fh:
    for line in fh:
        values = line.split("/")
        values[-1] = values[-1].split("\n")[0]
        data += [values[1:], ]

In [29]:
for i in range(len(data)):
        data[i] = list(combinations(data[i], 2))

In [30]:
tups = [i for s in data for i in s]

In [31]:
from collections import Counter

In [32]:
ranking = list(zip(Counter(tups).keys(), Counter(tups).values())) #Check!

In [33]:
DG = nx.DiGraph()

In [34]:
for i in range(len(ranking)):
    DG.add_edge(*ranking[i][0][::-1], weight=ranking[i][1])

In [35]:
list(DG.nodes())

['Morgan Freeman',
 'Tim Robbins',
 'Bob Gunton',
 'William Sadler',
 'Clancy Brown',
 'Gil Bellows',
 'Mark Rolston',
 'James Whitmore',
 'Jeffrey DeMunn',
 'Larry Brandenburg',
 'Neil Giuntoli',
 'Brian Libby',
 'David Proval',
 'Joseph Ragno',
 'Jude Ciccolella',
 'Paul McCrane',
 'Renee Blaine',
 'Scott Mann',
 'John Horton',
 'Gordon Greene',
 'Alfonso Freeman',
 'V.J. Foster',
 'John E. Summers',
 'Frank Medrano',
 'Mack Miles',
 'Alan R. Kessler',
 'Morgan Lund',
 'Cornell Wallace',
 'Gary Lee Davis',
 'Neil Summers',
 'Ned Bellamy',
 'Joe Pecoraro',
 'Harold E. Cope Jr.',
 'Brian Delate',
 'Don McManus',
 'Donald Zinn',
 'Dorothy Silver',
 'Robert Haley',
 'Dana Snyder',
 'John D. Craig',
 'Ken Magee',
 'Eugene C. DePasquale',
 'Bill Bolender',
 'Ron Newell',
 'John R. Woodward',
 'Chuck Brauchler',
 'Dion Anderson',
 'Claire Slemmer',
 'James Kisicki',
 'Rohn Thomas',
 'Charlie Kearns',
 'Rob Reider',
 'Brian Brophy',
 'Paul Kennedy',
 'James Babson',
 'Dennis Baker',
 'Fred C

In [36]:
list(DG.edges())

[('Morgan Freeman', 'Tim Robbins'),
 ('Morgan Freeman', 'Christian Bale'),
 ('Morgan Freeman', 'Heath Ledger'),
 ('Morgan Freeman', 'Aaron Eckhart'),
 ('Morgan Freeman', 'Michael Caine'),
 ('Morgan Freeman', 'Maggie Gyllenhaal'),
 ('Morgan Freeman', 'Gary Oldman'),
 ('Morgan Freeman', 'Tom Hardy'),
 ('Morgan Freeman', 'Joseph Gordon-Levitt'),
 ('Morgan Freeman', 'Anne Hathaway'),
 ('Morgan Freeman', 'Marion Cotillard'),
 ('Morgan Freeman', 'Liam Neeson'),
 ('Morgan Freeman', 'Katie Holmes'),
 ('Morgan Freeman', 'Cillian Murphy'),
 ('Morgan Freeman', 'Tom Wilkinson'),
 ('Morgan Freeman', 'Rutger Hauer'),
 ('Morgan Freeman', 'Ken Watanabe'),
 ('Morgan Freeman', 'Mark Boone Junior'),
 ('Morgan Freeman', 'Linus Roache'),
 ('Morgan Freeman', 'Clint Eastwood'),
 ('Morgan Freeman', 'Gene Hackman'),
 ('Morgan Freeman', 'Hilary Swank'),
 ('Bob Gunton', 'Tim Robbins'),
 ('Bob Gunton', 'Morgan Freeman'),
 ('William Sadler', 'Tim Robbins'),
 ('William Sadler', 'Morgan Freeman'),
 ('William Sadler'

In [37]:
ranking = nx.pagerank(DG, alpha=0.7)

In [38]:
import operator
sorted(ranking.items(), key=operator.itemgetter(1), reverse=True) #OK!

[('Leonardo DiCaprio', 0.005301581153586184),
 ('Robert De Niro', 0.0031767750594758606),
 ('Tom Hanks', 0.002706939278363638),
 ('Jamie Foxx', 0.002674938533263204),
 ('Al Pacino', 0.0026067895631016642),
 ('Christoph Waltz', 0.0023869428327710227),
 ('Christian Bale', 0.0023457037276863996),
 ('Ben Kingsley', 0.002126442144065199),
 ('Ralph Fiennes', 0.001985591122045252),
 ('Matt Damon', 0.0019347360634161034),
 ('Morgan Freeman', 0.0019147996660502042),
 ('Tom Hardy', 0.001903507532098764),
 ('Liam Neeson', 0.001886379384088311),
 ('Brad Pitt', 0.0018378618163891014),
 ('Gary Oldman', 0.0017609178290265638),
 ('Harrison Ford', 0.0016935315930430726),
 ('Ryan Gosling', 0.0016011964474452324),
 ('Kevin Spacey', 0.0015022697887060819),
 ('James Stewart', 0.0014937345783668245),
 ('Clint Eastwood', 0.0014855238184675336),
 ('Diahnne Abbott', 0.0014765700799090032),
 ('Michael Caine', 0.001473672688941038),
 ('Antonella Attili', 0.0014417468054251982),
 ('Jack Nicholson', 0.001382863784